# Inference and Save Model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch
import os

peft_model_path = '../models/amz_movie_tv_distilgpt2_1k'

config = PeftConfig.from_pretrained(peft_model_path)
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
model = PeftModel.from_pretrained(model, peft_model_path)
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [2]:
text = 'I have to'

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], pad_token_id=tokenizer.pad_token_id, max_new_tokens=10)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I have to be careful about what I'm saying. I'm


In [3]:
# upload to huggingface hub
from huggingface_hub import notebook_login

notebook_login()

In [4]:
model.push_to_hub(peft_model_path.split('/')[-1], use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/neilsun2009/amz_movie_tv_distilgpt2_1k/commit/832c29175be61aa0b03715a9143cb60a081259fe', commit_message='Upload model', commit_description='', oid='832c29175be61aa0b03715a9143cb60a081259fe', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
# hugging face hosted inference API
import requests
import os


API_URL = f"https://api-inference.huggingface.co/models/neilsun2009/{peft_model_path.split('/')[-1]}"
# API_URL = f"https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": f"Bearer !"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "I think tha",
    # these params actually are not useable for PEFT models
    "parameters": {
        "return_full_text": False,
        "temperature": 5.0,
        "repetition_penalty": 10.0,
        "top_k": 5
    }
})

print(output)

environ({'USER': 'azureuser', 'SSH_CLIENT': '210.3.146.60 4729 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '1', 'MOTD_SHOWN': 'pam', 'HOME': '/home/azureuser', 'OLDPWD': '/home/azureuser', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'LOGNAME': 'azureuser', '_': '/data/conda/env/bogo/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '2', 'PATH': '/data/conda/env/bogo/bin:/home/azureuser/miniconda3/condabin:/home/azureuser/.vscode-server/bin/6a995c4f4cc2ced6e3237749973982e751cb0bf9/bin/remote-cli:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT_FOLDER': '/home/azureuser/.vscode-server', 'XDG_RUNTIME_DIR': '/run/user/1000', 'LANG': 'C.UTF-8', 'SHELL': '/usr/bin/zsh', 'PWD': '/home/azureuser', 'SSH_CONNECTION': '210.3.146.60 4729 10.0.0.4 22', 'VSCODE_HANDLES_SIGPIPE': 'true', 'ZSH': '/home/azureuser/.oh-my-zsh', 'PAGER': 'cat', 'LESS': '-R', 'LSCOLORS': 'Gxfxcxdxbxegedabagacad', 'LS_COLORS': '', 'VSCO

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_242975/1589239075.py:9 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_242975/1589239075.py'                       │
│ <frozen os>:679 in __getitem__                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'HF_BEARER'